# The G In RAG

In [0]:
dbutils.widgets.text("query", "")
query = dbutils.widgets.get("query")

dbutils.widgets.text("user", "")
user = dbutils.widgets.get("user")

In [0]:
%pip install openai
%restart_python

In [0]:
# ------------------------------------------
# MCP Step 1: Create MCP Context Metadata
# ------------------------------------------
from datetime import datetime
import uuid

def generate_mcp_context(user: str, query: str):
    return {
        "request_id": str(uuid.uuid4()),
        "timestamp": datetime.utcnow().isoformat(),
        "user": user,
        "query": query,
    }

In [0]:
# user = "amitesh" 
# query = "What is the candidates name?"

# MCP context generation
mcp_context = generate_mcp_context(user=user, query=query)

In [0]:
from databricks.vector_search.client import VectorSearchClient

# Initialize client
vsc = VectorSearchClient()

# Get index
index = vsc.get_index(index_name="docai-dbx.gold.doc_embeddings_index")

# Use the built-in embedding function for querying (same one used for index creation)
results = index.similarity_search(
    query_text="What is the refund policy?",
    columns=["chunk_id", "file_name", "text_chunk"],
    num_results=5
)
#display(results)

In [0]:
from openai import OpenAI
import os

# Use token from current notebook session
DATABRICKS_TOKEN = dbutils.notebook.entry_point.getDbutils().notebook().getContext().apiToken().get()

# Init client with Databricks endpoint as base_url
client = OpenAI(
    api_key=DATABRICKS_TOKEN,
    base_url="<!workspace>" 
)

# Construct context from vector search
if isinstance(results["result"], list):
    context = "\n\n".join([r["text_chunk"] for r in results["result"]])
else:
    context = results["result"]
query = "what is the name of the applicant?"

# Ask the LLM
chat_completion = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "You are a helpful assistant. Answer questions based on the document excerpts provided."},
        {"role": "user", "content": f"""Use the following document to answer the question.

Document:
{context}

Question: {query}
"""}
    ],
    model="databricks-llama-4-maverick",  # or another model you deployed
    max_tokens=300
)

final_answer = chat_completion.choices[0].message.content
# Print answer
print(final_answer)

In [0]:
mcp_result = {
    "request_id": mcp_context["request_id"],
    "timestamp": mcp_context["timestamp"],
    "user": mcp_context["user"],
    "query": mcp_context["query"],
    "answer": final_answer,
    "context":context
}

from pyspark.sql import Row
from pyspark.sql.types import StructType, StructField, StringType, TimestampType
from datetime import datetime

# Ensure timestamp is in datetime format
mcp_result["timestamp"] = datetime.fromisoformat(mcp_result["timestamp"])

# Define schema
schema = StructType([
    StructField("request_id", StringType(), True),
    StructField("timestamp", TimestampType(), True),
    StructField("user", StringType(), True),
    StructField("query", StringType(), True),
    StructField("answer", StringType(), True),
    StructField("context", StringType(), True)  # Assuming context is a string
])

# Convert to Row and append
row = Row(**mcp_result)
df = spark.createDataFrame([row],schema=schema)

df.write.mode("append").format("delta").saveAsTable("`docai-dbx`.mcp_audit.audit_logs")